In [1]:
!pip install -r requirements.txt -q
!pip install google-api-python-client -q

In [2]:
import openai
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.schema.messages import SystemMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import(
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
import time  # Time module for handling time-related operations

In [3]:
import os
os.environ['OPENAI_API_KEY'] = "your_api_key"

In [4]:
import pandas as pd
df = pd.read_csv('Test.csv', encoding='latin1')
df["evaluation"] = ""
df.head()

,id,problem,solution,evaluation
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,


In [5]:
df.tail()

,id,problem,solution,evaluation
15,16,The fashion industry contributes about 10% of ...,"""""Full Turn Fashion"""" is a comprehensive circu...",
16,17,Rapid advancement in technology and our demand...,We can contribute to the circular economy by d...,
17,18,Reducing plastic use in the food industry is a...,"Building on the Green Box Project, a digital s...",
18,19,"In a fast-paced, consumer-driven society, elec...","To address this issue, businesses can transiti...",
19,20,The fast fashion industry promotes high consum...,I propose an innovative digital platform that ...,


In [6]:
llm = ChatOpenAI(temperature = 0, model_name = "gpt-4")

In [7]:
system_message = """Objective: As an expert in renewable energy, the green economy, and circular economy business opportunities, evaluate individual business ideas focusing on circular economy solutions. 
Assess the potential impact, viability, and alignment with circular economy principles, considering both the {problem} and {solution} provided. Do not make up or add anything to the {solution}.

Important: Start all generations with the Sentiment and the Category as described further in this prompt.
Important: If the provided information does not contain a specific business idea focusing on circular economy solutions terminate the evaluation and assign the sentiment: Not Applicable.

Instructions:
1. Evaluate each {problem} and {solution} pair based on the following six criteria:
    (a) Circularity Potential: Assess the idea's effectiveness in promoting circularity, reducing waste, enhancing resource efficiency, or encouraging reuse and recycling.
                IMPORTANT: If the solution does not promote directly circularity terminate the evaluation and assign the sentiment: Not Applicable. If the solution directly promotes circularity, continue to follow the remaining instructions provided in this prompt.
    (b) Scalability: Evaluate the potential for the business model's broader adoption, replication, and impact across different contexts or regions.
    (c) Innovation and Novelty: Determine the level of innovation, novelty, and disruptive potential introduced by the idea.
    (d) Economic Viability: Analyze the economic prospects, revenue generation, profitability, and investment attractiveness of the proposed business opportunity.
    (e) Social and Environmental Impact: Assess the idea's contributions to sustainability goals, environmental conservation, and social welfare.
    (f) Feasibility and Implementation: Evaluate the technical feasibility, readiness for deployment, and regulatory compliance of the proposed business opportunity.
    
2. If all criterion were evaluated, provide scores for each criterion on a scale of 1 to 10, considering the alignment of the idea with circular economy principles and its overall potential. 

3. Final Evaluation: Provide a summary evaluation for the submitted idea, highlighting strengths, areas for improvement, and overall assessment. If a sentiment of Not Appicable was assigned, provide an explanation of why.

4. Consolidate the scores assigned to each criterion to generate an overall assessment score on a scale of 1 to 60.

5. Generate a sentiment using one word based on the following:
    (a) Positive: if the consolidated score is greater than 30 out of 60
    (b) Negative: if the consolidated score is equal to or less than 30 out of 60
    (c) Not Applicable: the solution does not directly promote circular principles or if there is no specific business idea outlined in the solution
    
6. Generate a Category into which the idea falls based on the below:
    (a) Design for Circularity: 
        (i) Focuses on designing products, services, and systems with circularity in mind.
        (ii) Emphasizes durability, reparability, modularity, and recyclability to extend product lifecycles and minimize waste.

    (b) Resource Efficiency and Optimization:
        (i) Aims to maximize the use of resources, minimize waste generation, and optimize resource consumption across the value chain.
        (ii) Includes practices such as waste reduction, energy efficiency, water conservation, and material optimization.

    (c) Reuse and Repair:
        (i) Encourages the reuse, refurbishment, and repair of products, components, and materials.
        (ii) Promotes initiatives such as remanufacturing, refurbishing, repurposing, and extending product lifecycles through maintenance and repair services.

    (d) Recycling and Material Recovery:
        (i) Focuses on recycling, recovering, and reintegrating materials and components back into the production cycle.
        (ii) Includes initiatives such as material recovery, recycling technologies, closed-loop systems, and sustainable waste management practices.

    (e) Circular Business Models:
        (i) Explores innovative business models that prioritize circularity, sustainability, and value creation.
        (ii) Includes models such as product-as-a-service, sharing and collaborative consumption, leasing, subscription models, and circular supply chains.

    (f) Sustainable Supply Chains:
        (i) Emphasizes creating resilient, transparent, and sustainable supply chains that promote ethical sourcing, responsible production, and circularity.
        (ii) Includes practices such as sustainable sourcing, responsible procurement, eco-friendly logistics, and supply chain optimization.

    (g) Consumer Engagement and Behavior Change:
        (i) Focuses on raising awareness, educating consumers, and promoting sustainable consumption and lifestyle choices.
        (ii) Includes initiatives such as consumer awareness campaigns, eco-labeling, sustainable packaging, and promoting responsible consumption patterns.

    (h) Policy and Regulatory Frameworks:
        (i) Advocates for the development and implementation of supportive policies, regulations, incentives, and frameworks that facilitate the transition to a circular economy.
        (ii) Includes initiatives such as extended producer responsibility (EPR), eco-design standards, waste management regulations, and circular economy policies at local, regional, and national levels.
              
"""

In [8]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_message)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [9]:
results = []

for index, row in df.iterrows():
    problem = row['problem']
    solution = row['solution']
    
    # Call the OpenAI API
    prompt = chat_prompt.format_prompt(
        problem = problem,
        solution = solution
    ).to_messages()
       
    response = llm(prompt)
    results.append(response.content)
    
    time.sleep(0.5)

# Create a new column in the dataframe with the results
df['evaluation'] = results

In [10]:
df

,id,problem,solution,evaluation
0,1,The construction industry is indubitably one o...,"Herein, we propose an innovative approach to m...",Sentiment: Positive\nCategory: Design for Circ...
1,2,"I'm sure you, like me, are feeling the heat - ...","Imagine standing on a green hill, not a single...",Sentiment: Not Applicable\nCategory: Not Appli...
2,3,The massive shift in student learning towards ...,"Implement a """"Book Swap"""" program within educa...",Sentiment: Positive\nCategory: Reuse and Repai...
3,4,The fashion industry is one of the top contrib...,The proposed solution is a garment rental serv...,Sentiment: Positive\nCategory: Circular Busine...
4,5,The majority of the materials used in producin...,An innovative concept would be a modular elect...,Evaluation:\n\n(a) Circularity Potential: The ...
5,6,Businesses worldwide expend substantial financ...,The proposed solution involves developing a se...,Evaluation:\n\n(a) Circularity Potential: The ...
6,7,more than 130 Billon plastic bottles waste ann...,Bariq factory to recyle plastic bottels,Sentiment: Positive\nCategory: Recycling and M...
7,8,"In congested cities like Berlin, one of the si...",Let's revolutionize the carsharing experience...,Sentiment: Positive\nCategory: Circular Busine...
8,9,One major global issue we face today is the su...,"My solution is an innovative Reloop - System, ...",1. Evaluation:\n (a) Circularity Potential:...
9,10,The usage of plastic bottles,"Creating a service that sells bottles, and re-...",Sentiment: Positive\nCategory: Circular Busine...


In [11]:
df.to_csv('test_update.csv')

In [12]:
prompt = chat_prompt.format_prompt(
    problem= "The construction industry is indubitably one of the significant contributors to global waste, contributing approximately 1.3 billion tons of waste annually, exerting significant pressure on our landfills and natural resources. Traditional construction methods entail single-use designs that require frequent demolitions, leading to resource depletion and wastage.",
    solution = "Herein, we propose an innovative approach to mitigate this problem: Modular Construction. This method embraces recycling and reuse, taking a significant stride towards a circular economy.   Modular construction involves utilizing engineered components in a manufacturing facility that are later assembled on-site. These components are designed for easy disassembling, enabling them to be reused in diverse projects, thus significantly reducing waste and conserving resources.  Not only does this method decrease construction waste by up to 90%, but it also decreases construction time by 30-50%, optimizing both environmental and financial efficiency. This reduction in time corresponds to substantial financial savings for businesses. Moreover, the modular approach allows greater flexibility, adapting to changing needs over time.  We believe, by adopting modular construction, the industry can transit from a 'take, make and dispose' model to a more sustainable 'reduce, reuse, and recycle' model, driving the industry towards a more circular and sustainable future. The feasibility of this concept is already being proven in markets around the globe, indicating its potential for scalability and real-world application."
).to_messages()

In [13]:
result = llm(prompt)
print(result.content)

Sentiment: Positive
Category: Design for Circularity, Resource Efficiency and Optimization, Circular Business Models

Evaluation:

(a) Circularity Potential: The proposed idea of modular construction aligns well with the principles of circular economy. By designing components for easy disassembly and reuse, it promotes resource efficiency and waste reduction. The potential for reducing construction waste by up to 90% is significant. Score: 9/10

(b) Scalability: The modular construction approach is highly scalable. It can be adopted in various contexts and regions, given the universal nature of construction activities. The fact that it is already being implemented in markets globally is a positive indicator. Score: 8/10

(c) Innovation and Novelty: While modular construction is not a new concept, its application in promoting circularity in the construction industry is innovative. The idea disrupts traditional construction methods that are wasteful and resource-intensive. Score: 7/10

(

In [14]:
#agent = create_pandas_dataframe_agent(
##    llm,
#    df,
#    verbose=True,
#    agent_type = AgentType.OPENAI_FUNCTIONS,
#    max_iterations=6,
#    early_stopping_method="generate",
#    agent_kwargs={"system_message": system_message}
#)